# PHÂN TÍCH DỮ LIỆU, ĐẶT VÀ TRẢ LỜI CÂU HỎI

## 1. Import thư viện cần thiết

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## 2. Đọc dữ liệu từ file csv

In [ ]:
anime23_df = pd.read_csv('anime23_preprocessing.csv')
anime23_df.head()

## 3. Đặt câu hỏi

### Câu 1

#### Những studio sản xuất được nhiều phim có đánh giá cao có đặc điểm gì?

- Rất nhiều chương trình mà ai đó có xu hướng thích hoặc có điểm số tốt đều đến từ cùng một studio, do đó ta nên xem xét chúng.
- Bắt đầu bằng cách lọc để chỉ xem các hãng phim anime hàng đầu (các studio sản xuất được nhiều bộ phim có `Rank` cao).
- Trả lời câu hỏi này sẽ giúp những người đang tìm kiếm những bộ phim anime chất lượng để xem có thêm gợi ý để tìm kiếm dựa trên các studio hàng đầu.
- Trả lời câu hỏi này sẽ giúp chúng ta biết được đặc điểm của những studio thành công và khác biệt của họ trong ngành.

### Câu 2

### Câu 3

## 4. Trả lời câu hỏi

### Câu 1

#### Những studio sản xuất được nhiều phim có đánh giá cao có đặc điểm gì?

Trả lời câu hỏi trên sẽ giúp chúng ta tìm ra được những studio thành công và đặc điểm của họ.

Trả lời câu hỏi trên cũng giúp những người đang tìm kiếm anime hay để xem sẽ có những gợi ý rõ ràng hơn để tìm kiếm phim xem. Đặc biệt là đối với những người mới bắt đầu xem anime, họ có thể lựa chọn các anime đến từ những studio được đánh giá cao.

Để trả lời cho câu hỏi trên, chúng ta cần lấy ra bộ dữ liệu gồm các anime top đầu. Nhóm em quyết định lấy 1000 anime xếp theo `Rank` trong bộ dữ liệu (gần bằng 5% tổng số anime).

Đầu tiên, vì cột Studios là kiểu chuỗi, nên phải split nó ra và explode để mỗi cột chỉ có một giá trị của Studio.

In [ ]:
ques1_df = anime23_df
ques1_df['Studios'] = ques1_df['Studios'].str.split(', ')
studios_explode = ques1_df.explode('Studios')
#Loại bỏ các Studios là Unknown
studios_explode = studios_explode[studios_explode['Studios'] != 'UNKNOWN']
studios_explode.head()
#Xem thử kết quả sau khi explode

Tiếp theo, ta phải lọc các bộ anime thỏa mãn điều kiện.

In [ ]:
ques1_df = studios_explode[studios_explode['Rank']!=0]
ques1_df = ques1_df[ques1_df['Rank']<=1000]

Tiếp theo, cần tính số bộ phim tốt của mỗi studio.

In [ ]:
top_anime_count = ques1_df.groupby('Studios')['Score'].count().sort_values(ascending = False)

Lọc dữ liệu để chỉ lấy 15 studio có nhiều anime rank cao nhất

In [ ]:
top_studios = top_anime_count.nlargest(15)

Như vậy ta đã tìm ra được 15 studio sản xuất được nhiều bộ phim có đánh giá tốt cùng với số bộ phim tốt được sản xuất là:

In [ ]:
top_studios

#### Vẽ biểu đồ thể hiện tỉ lệ anime top đầu của các studio

In [ ]:
data = top_studios.copy()
# Gộp các studio còn lại vào nhóm "Others"
data['Others'] = ques1_df['Studios'].value_counts().sum() - top_studios.sum()
# Vẽ biểu đồ tròn
plt.figure(figsize=(10, 10))
plt.pie(data, labels=data.index, autopct='%1.1f%%', startangle=90, textprops={'fontsize': 10})
plt.title("Anime Studio's in Top 1000", fontsize=14)
plt.show()

Nhận xét:
- Madhouse và Production I.G là hai studio có số lượng anime nhiều nhất trong top 1000 anime được đánh giá cao nhất, đều chiếm tỉ lệ 6.0%.
- Hơn một nửa số bộ phim được đánh giá cao nhất thuộc về top 15 studio hàng đầu trên tổng số hơn 1000 studio.
- Nhìn chung giữa các studio top đầu không có cách biệt quá lớn về số lượng bộ phim hay được sản xuất.
- Tất cả các xưởng phim hàng đầu trong top 15 đều đến từ Nhật Bản.

**Tiếp theo ta sẽ khám phá thêm về đặc điểm của các studio thành công. Để có cái nhìn tổng quan về hoạt động của các studio thì ta sẽ phân tích dựa trên bộ dữ liệu ban đầu có đầy đủ các anime là studios_explode.**

#### Dạng phim mà các studio trên sản xuất là phim truyền hình, anime dài tập hay movie,...?

Thực hiện thống kê về số lượng anime của mỗi loại (Type) cho từng studio

In [ ]:
studios_list = top_studios.index
filtered_data = studios_explode[studios_explode['Studios'].isin(studios_list)]
type_counts = filtered_data.groupby(['Studios', 'Type']).size().reset_index(name='Count')
type_counts

Vẽ biểu đồ phân bố các dạng phim anime theo từng studio

In [ ]:
# Vẽ biểu đồ scatterplot
fig, ax = plt.subplots(figsize=(10, 5))
sns.scatterplot(x='Studios', y='Count', hue='Type', data=type_counts, ax=ax)
plt.xticks(rotation=90)
plt.title('Distribution of anime types by each studio')
plt.legend(bbox_to_anchor=(1, 1))
# Hiển thị biểu đồ
plt.show()

Nhận xét:
- Các studio đều sản xuất phim hoạt hình đa dạng, nhưng phim truyền hình (TV) là dạng phim được hầu hết các studio tập trung sản xuất nhất. Lí do có thể là vì TV là dạng dễ sản xuất và được yêu thích nhất đối với người xem. OVA và Movive cũng là 2 dạng phim phổ biến và được sản xuất nhiều.
- Các studio có số lượng anime TV nhiều nhất là Toei Animation, Sunrise và J.C.Staff. Đây là các studio lớn và có lịch sử lâu đời trong ngành anime, với nhiều anime nổi tiếng như Dragon Ball, One Piece, Gundam, Code Gea Revolutionary Girl Utena,... Đặc biệt, Toei Animation là studio lớn, có lượng sản xuất hàng đầu và cũng sản xuất nhiều Movie và OVA anime nhất.
- Các studio sản xuất nhiều anime Movie là Madhouse, Production I.G và TMS Entertainment. Đây là các studio có kỹ năng vẽ hoạt họa cao và có nhiều bộ phim hoạt hình đoạt giải thưởng quốc tế, như Paprika, Wolf Children, The Girl Who Leapt Through Time, Redline, Ghost in the Shello,...
- Các studio sản xuất nhiều anime OVA là Studio Deen, J.C.Staff và Pierrot. Đây là các studio có nhiều anime dài tập và có nhiều phần ngoại truyện, như Rurouni Kenshin, Higurashi no Naku Koro ni, Umineko no Naku Korlaye,...
- Các studio MAPPA, WIT Studio và ufotable sản xuất nhiều anime ONA. Đây là các studio mới và có nhiều anime được phát hành trên các nền tảng trực tuyến, như Netflix, Amazon Prime và Crunchyroll. Một số anime của họ là Yuri!!! on Ice, Inuyashiki, Dororo, Vinland Saga, Attack on Titan, Fate/Zero, Fate/stay night: Unlimited Blade Wo,...
- Các studio Kyoto Animation, Shaft và A-1 Pictures sản xuất khá nhiều anime Special. Đây là các studio có nhiều anime hài hước và lãng mạn, và có nhiều tập đặc biệt để tăng thêm sự gắn kết với khán giả, như K-On!, Clannad, The Melancholy of Haruhi Suzumiya, Monogatari Series, Puella Magi Madoka Magica, Sword Art Online, Fairy ,...

#### Các studio có xu hướng chuyển thể như thế nào?

Thực hiện thống kê về số lượng anime của mỗi Source cho từng studio

In [ ]:
source_counts = filtered_data.groupby(['Studios', 'Source']).size().reset_index(name='Count')
source_counts

Vẽ biểu đồ phân bố các nguồn chuyển thể theo từng studio

In [ ]:
# Vẽ biểu đồ scatterplot
fig, ax = plt.subplots(figsize=(10, 5))
sns.scatterplot(x='Studios', y='Count', hue='Source', data=source_counts, ax=ax)
plt.xticks(rotation=90)
plt.title('Distribution of anime source by each studio')
plt.legend(bbox_to_anchor=(1, 1))
# Hiển thị biểu đồ
plt.show()

Nhận xét:
- Các studio lớn như Production I.G, J.C.Staff, MADHOUSE và Toei Animation có số lượng anime chuyển thể nhiều nhất, từ các nguồn đa dạng như manga, light novel, game, v.v. Điều này cho thấy họ có khả năng thích ứng với nhiều thể loại và phong cách khác nhau, cũng như có mối quan hệ tốt với các nhà xuất bản và nhà phát triển game.
- Các studio nhỏ hơn như Kyoto Animation, Shaft, ufotable và Wit Studio có số lượng anime chuyển thể ít hơn, nhưng chủ yếu là từ các nguồn riêng biệt như light novel, visual novel, web manga, v.v. Điều này cho thấy họ có sự chọn lọc và tập trung cao trong việc chuyển thể các tác phẩm có chất lượng và độc đáo, cũng như có phong cách nghệ thuật đặc trưng và nhận dạng.
- Các studio khác như Bones, MAPPA, Pierrot, A-1 Pictures và Sunrise có số lượng anime chuyển thể trung bình, từ các nguồn khác nhau, nhưng có xu hướng nhiều hơn về các thể loại như hành động, phiêu lưu, khoa học viễn tưởng, v.v. Điều này cho thấy họ có kinh nghiệm và năng lực trong việc chuyển thể các tác phẩm có yếu tố kịch tính và hấp dẫn, cũng như có kỹ thuật hoạt họa và hiệu ứng cao.

#### Sự tương quan giữa số lượng và chất lượng

Đầu tiên cần tính tổng số bộ phim mà mỗi studio đã sản xuất.

In [ ]:
total_anime = filtered_data['Studios'].value_counts()
print(total_anime)

Tiếp theo, cần tính điểm trung bình các bộ phim của mỗi studio.

In [ ]:
filtered_data = filtered_data.dropna(subset=['Score'])
avg_score_by_studio = filtered_data.groupby("Studios")['Score'].mean()
avg_score_by_studio

Cuối cùng, vẽ biểu đồ.

In [ ]:
total_anime = total_anime.sort_index()
avg_score_by_studio = avg_score_by_studio.sort_index()
#Vẽ biểu đồ
fig, ax = plt.subplots(figsize = (15,7))

ax1 = total_anime.plot(kind='bar', rot = 90, label = 'Số anime')
ax2 = avg_score_by_studio.plot(secondary_y=True, rot = 90, color = 'r', label = 'Điểm trung bình')
ax2.set_ylim(5, 9)

#Vẽ 2 legend trên cùng 1 bảng
h1, l1 = ax.get_legend_handles_labels()
h2, l2 = ax.right_ax.get_legend_handles_labels()
handles = h1+h2
labels = l1+l2
ax.legend(handles, labels, loc='upper left')

#Thêm title và xlabel, ylabel

plt.title("Biểu đồ thể hiện tổng số anime đã sản xuất và số điểm trung bình của mỗi studio")
ax1.set_ylabel("Số anime")
ax2.set_ylabel("Điểm trung bình")

plt.show()

Nhận xét:
- Một số studio có số lượng anime sản xuất thấp nhưng điểm trung bình cao như Kyoto Animation, ufotable, Wit Studio, v.v. Điều này có thể do họ chọn lọc và tập trung vào các tác phẩm có chất lượng và độc đáo, hoặc do họ có phong cách nghệ thuật đặc trưng và nhận dạng.
- Một số studio có số lượng anime sản xuất cao nhưng điểm trung bình thấp hơn như Toei Animation, Pierrot, J.C.Staff, v.v. Điều này có thể do họ chuyên về các thể loại phổ biến nhưng ít sáng tạo, do họ không đầu tư nhiều vào kỹ thuật hoạt họa và nội dung hoặc do sự chênh lệch giữa chất lượng các sản phẩm. Toei Animation có xếp hạng trung bình thấp nhất với 6.623422 trên 10, mặc dù họ là studio có số lượng sản xuất lớn nhất.
- Có một số studio có số lượng anime sản xuất và điểm trung bình ở mức trung bình, ví dụ như Bones, Studio Deen, Production I.G., v.v. Điều này có thể do họ có sự cân bằng giữa số lượng và chất lượng, hoặc do họ có nhiều thể loại và phong cách khác nhau.
- Tuy nhiên, có thể thấy rằng số lượng và chất lượng anime của các studio không có mối tương quan cố định, còn mà phụ thuộc vào nhiều yếu tố khác nhau.

#### Vẽ boxplot thể hiện điểm trung bình các phim của mỗi studio

In [ ]:
# Create a boxplot using Seaborn
plt.figure(figsize=(12, 8))
sns.set_theme(style="whitegrid")
sns.boxplot(x='Studios', y='Score', data=filtered_data, hue='Studios', palette='viridis', dodge=False, legend=False)
plt.title('Boxplot of Top 15 Anime Studios by Average Score')
plt.xticks(rotation=90)
plt.show()

Nhận xét:
- Biểu đồ này cho ta cái nhìn sâu sắc hơn về chất lượng các tác phẩm của mỗi studio.
- Vì đây là top các studio hàng đầu nên chất lượng sản xuất anime của các studio khá đồng đều, với điểm trung bình nằm trong khoảng từ 6 đến 8. Tuy nhiên, cũng có một số outliers, cho thấy có những phim xuất sắc hoặc kém chất lượng.
- Các studio như Studio Deen, Production I.G và Shaft có phạm vi điểm rộng dù số bộ phim họ sản xuất ở mức trung bình và thấp, cho thấy sự chênh lệch lớn trong chất lượng sản phẩm của họ.
- Có thể thấy rằng các studio thành công này không sản xuất nhiều tác phẩm nhất hay có mức điểm trung bình cao nhất. Có thể nói rằng chất lượng công việc mà các studio này thực hiện mới khiến họ có được rất nhiều chương trình lọt vào top 1000 với điểm số tốt.

### Câu 2

### Câu 3

# PHẢN ÁNH

## 1. Bạn đã gặp những khó khăn gì?

### Hân

- Bước đặt câu hỏi là bước khó nhất sao cho câu hỏi hợp lý, có giá trị và không bị mơ hồ.
- Bước đầu phải xử lý lại bộ dữ liệu vì cột Studios có nhiều giá trị.
- Xác định tiêu chuẩn hợp lý cho các bộ phim tốt để lọc bộ dữ liệu.
- Xác định những cột dữ liệu cần thiết để trả lời câu hỏi, khó khăn trong việc khám phá và tìm ra các thống kê nổi bật để trả lời câu hỏi. Khó khăn trong việc chọn những phân tích và thống kê hợp lý nhất để trả lời cho câu hỏi của mình.
- Nhận xét khách quan và có giá trị dựa trên dữ liệu và thống kê.

### Khuê

### Tuấn

## 2. Bạn đã học được những gì

### Hân

- Đánh giá được ý nghĩa và giá trị của các bộ dữ liệu. Biết một bộ dữ liệu như thế nào là tốt và phù hợp.
- Có một số kinh nghiệm về tiền xử lý dữ liệu.
- Đặt được các câu hỏi có ý nghĩa về mật dữ liệu.
- Trình bày một báo cáo phân tích dữ liệu hoàn chỉnh.

### Khuê

### Tuấn

## 3. Nếu có nhiều thời gian hơn bạn sẽ làm gì?

- Thống kê thêm về phân bố Rating, Genres của các top-studio.
- Phân tích sự tương quan giữa các yếu tố như số lượng người xem, số lượng người yêu thích và điểm số của các studio. Phân tích mức độ phổ biến và mức độ tương tác của người xem.
- Phân tích sự ảnh hưởng của các studio đối với nhau. Tìm ra các studio có cùng nhà sản xuất, các studio đồng sản xuất phim... Điều này sẽ giúp hiểu được sự hợp tác và cạnh tranh giữa các studio.
